#### Required installations

In [ ]:
!pip install gensim
!pip install git+https://github.com/boudinfl/pke.git
!python -m spacy download en
!pip install bert-extractive-summarizer --upgrade --force-reinstall

!pip install -U nltk
!pip install -U pywsd

In [3]:
!pip install spacy==3.2.3 --upgrade --force-reinstall

     |████████████████████████████████| 6.0 MB 4.4 MB/s 
  Using cached preshed-3.0.6-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (125 kB)
  Using cached spacy_legacy-3.0.9-py2.py3-none-any.whl (20 kB)
  Using cached wasabi-0.9.1-py3-none-any.whl (26 kB)
  Using cached murmurhash-1.0.6-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21 kB)
  Using cached thinc-8.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (653 kB)
  Using cached cymem-2.0.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached spacy_loggers-1.0.2-py3-none-any.whl (7.2 kB)
  Using cached pathy-0.6.1-py3-none-any.whl (42 kB)
  Using cached catalogue-2.0.7-py3-none-any.whl (17 kB)
  Using cached blis-0.7.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.w

In [1]:
!pip install flashtext

In [2]:
!pip install pywsd

In [3]:
!pip install -U wn==0.0.22

#### Import libraries

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('popular')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Down

True

In [5]:
import pprint
import itertools
import re
import pke
import string
from nltk.corpus import stopwords

In [6]:
import requests
import json
import re
import random
from wn import WordNet
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn


Warming up PyWSD (takes ~10 secs)... took 6.93432879447937 secs.


In [7]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

In [8]:
from summarizer import Summarizer

#### Function definitions

In [9]:
def get_nouns_multipartite(text):
    out=[]

    extractor = pke.unsupervised.MultipartiteRank()
    # extractor = pke.unsupervised.TfIdf()
    #    not contain punctuation marks or stopwords as candidates.
    # pos = {'PROPN'}
    pos = {'VERB', 'ADJ', 'NOUN'}
    stoplist = list(string.punctuation)
    stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    stoplist += stopwords.words('english')

    extractor.load_document(input=text,language='en',normalization=None, stoplist=stoplist)
    extractor.candidate_selection(pos=pos)
    # 4. build the Multipartite graph and rank candidates using random walk,
    #    alpha controls the weight adjustment mechanism, see TopicRank for
    #    threshold/method parameters.
    extractor.candidate_weighting(alpha=1.1,
                                  threshold=0.75,
                                  method='average')
    keyphrases = extractor.get_n_best(n=20)

    for key in keyphrases:
        out.append(key[0])

    return out


In [10]:


def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    # Remove any short sentences less than 20 letters.
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences


In [11]:
# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

#### Actual code

In [20]:
model = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
def func(model, full_text):
  
  result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)

  summarized_text = ''.join(result)
  # print (summarized_text)

  keywords = get_nouns_multipartite(full_text) 
  # print (keywords)
  filtered_keys=[]
  for keyword in keywords:
      if keyword.lower() in summarized_text.lower():
          filtered_keys.append(keyword)
          
  # print (filtered_keys)

  sentences = tokenize_sentences(summarized_text)
  keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
          
  keyword_sentence_mapping

  key_distractor_list = {}

  for keyword in keyword_sentence_mapping:
      wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
      if wordsense:
          distractors = get_distractors_wordnet(wordsense,keyword)
          if len(distractors) ==0:
              distractors = get_distractors_conceptnet(keyword)
          if len(distractors) != 0:
              key_distractor_list[keyword] = distractors
      else:
          
          distractors = get_distractors_conceptnet(keyword)
          if len(distractors) != 0:
              key_distractor_list[keyword] = distractors


  index = 1
  result = []
  for each in key_distractor_list:
      sentence = keyword_sentence_mapping[each][0]
      pattern = re.compile(each, re.IGNORECASE)
      output = pattern.sub( " _______ ", sentence)
      choices = [each.capitalize()] + key_distractor_list[each]
      top4choices = choices[:4]

      new_entry = dict()
      new_entry['question']=output
      new_entry['ans']=each
      new_entry['distractor']=top4choices
      result.append(new_entry)
      index = index + 1

  return result

In [25]:
func(model, "Mike and Morris lived in the same village. While Morris owned the largest jewelry shop in the village, Mike was a poor farmer. Both had large families with many sons, daughters-in-law and grandchildren.")

[{'ans': 'daughters-in-law',
  'distractor': ['Daughters-in-law',
   'Brother-in-law',
   'Daughter-in-law',
   'Father-in-law'],
  'question': 'Both had large families with many sons,  _______  and grandchildren.'},
 {'ans': 'grandchildren',
  'distractor': ['Grandchildren', 'Baby', 'Bastard', 'Child'],
  'question': 'Both had large families with many sons, daughters-in-law and  _______ .'}]

In [ ]:


# f = open("egypt.txt","r")
# full_text = f.read()
# full_text = "One of the most famous people born in Warsaw was Maria Skłodowska-Curie, who achieved international recognition for her research on radioactivity and was the first female recipient of the Nobel Prize. Famous musicians include Władysław Szpilman and Frédéric Chopin. Though Chopin was born in the village of Żelazowa Wola, about 60 km (37 mi) from Warsaw, he moved to the city with his family when he was seven months old. Casimir Pulaski, a Polish general and hero of the American Revolutionary War, was born here in 1745."
full_text = "In Chapter 3 you learnt that woollen clothes are made from animal fibres.You also know that cotton clothesare made from plant fibres. We wear woollen clothes during winters when it is cold outside. Woollen clothes keep us warm. We prefer to wear light coloured cotton clothes when it is hot. These give us a feeling of coolness. You might have wondered why particular types of clothes are suitable for a particular season. In winter you feel cold inside the house. If you come out in the sun, you feel warm. In summer, you feel hot even inside the house. How do we know whether an object is hot or cold? How do we find out how hot or cold an object is? In this chapter we shall try to seek answers to some of these questions.In our day-to-day life, we come across a number of objects. Some of them are hot and some of them are cold. Tea is hot and ice is cold. List some objects you use commonly in Table 4.1. Mark these objects as hot or cold.We see that some objects are cold while some are hot. You also know that some objects are hotter than others while some are colder than others. How do we decide which object is hotter than the other? We often do it by touching the objects. But is our sense of touch reliable? Let us find out.A reliable measure of the hotness of an object is its temperature. Temperature is measured by a device called thermometer. Have you seen a thermometer? Recall that when you or someone else in your family had fever, the temperature was measured by a thermometer. The thermometer that measures our body temperature is called a clinical thermometer. Hold the thermometer in your hand and examine it carefully. If you do not have a thermometer, requesta friend to share it with you. A clinical thermometer looks like the one shown in Fig. 4.2. A clinical thermometer consists of a long, narrow, uniform glass tube. It has a bulb at one end. This bulb contains mercury. Outside the bulb, a small shining thread of mercury can be seen. If you do not see the mercury thread, rotate the thermometer a bit till you see it. You will also find a scale on the thermometer. The scale we use is the celsius scale, indicated by °C. A clinical thermometer reads temperature from 35°C to 42°C. Let us learn how to read a thermometer. First, note the temperature difference indicated between the two bigger marks. Also note down the number of divisions (shown by smaller marks) between these marks. Suppose the bigger marks read one degree and there are five divisions between them. Then, one small division can read 1/5 = 0.2°C. Wash the thermometer, preferably with an antiseptic solution. Hold it firmly and give it a few jerks. The jerks will bring the level of mercury down. Ensure that it falls below 35°C. Now place the bulb of the thermometer under your tongue. After one minute, take the thermometer out and note the reading. This is your body temperature. The temperature should always be stated with its unit, °C. The normal temperature of human body is 37°C. Note that the temperature is stated with its unit. The temperature of every person may not be 37°C. It could be slightly higher or slightly lower. Actually, what we call normal temperature is the average body temperature of a large number of healthy persons. The clinical thermometer is designed to measure the temperature of human body only. The temperature of human body normally does not go below 35oC or above 42oC. That is the reason that this thermometer has the range 35oC to 42oC. How do we measure the temperature of other objects? For this purpose, there are other thermometers. One such thermometer is known as the laboratory thermometer. The teacher will show you this thermometer. Look at it carefully and note the highest and the lowest temperature it can measure. The range of a laboratory thermometer is generally from –10°C to 110°C (Fig. 4.4). Also, as you did in the case of the clinical thermometer, find out how much a small division on this thermometer reads. You would need this information to read the thermometer correctly. You might have observed that a frying pan becomes hot when kept on a flame. It is because the heat passes from the flame to the utensil. When the pan is removed from the fire, it slowly cools down. Why does it cool down? The heat is transferred from the pan to the surroundings. So you can understand that in both cases, the heat flows from a hotter object to a colder object. In fact, in all cases heat flows from a hotter object to a colder object. The process by which heat is transferred from the hotter end to the colder end of an object is known as conduction. In solids, generally, the heat is transferred by the process of conduction. The materials which allow heat to pass through them easily are conductors of heat. For examples, aluminum, iron and copper. The materials which do not allow heat to pass through them easily are poor conductors of heat such as plastic and wood. Poor conductors are known as insulators. The water and air are poor conductors of heat."
model = Summarizer()
result = model(full_text, min_length=60, max_length = 500 , ratio = 0.4)

summarized_text = ''.join(result)
print (summarized_text)

In [ ]:
sentences = tokenize_sentences(summarized_text)
keyword_sentence_mapping = get_sentences_for_keyword(filtered_keys, sentences)
        
keyword_sentence_mapping

In [ ]:
key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

In [ ]:
index = 1
print ("#############################################################################")
print ("NOTE::::::::  Since the algorithm might have errors along the way, wrong answer choices generated might not be correct for some questions. ")
print ("#############################################################################\n\n")
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1